# Introduction

In this jupyter notebook we are performing entity matching using two tables named as "google" and "apple". These tables contains attributes related to games available on playstore and appstore. We are trying to match games present in these tables. We want to have a precision of atleast 90% and recall as high as possible.

First, we need to import py_entitymatching package and other libraries as follows:

In [87]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )


python version: 2.7.13 |Anaconda 4.3.0 (32-bit)| (default, Dec 20 2016, 23:08:16) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
pandas version: 0.19.2
magellan version: 0.1.0


Matching two tables typically consists of the following three steps:

<b>1. Reading the input tables</b>

<b>2. Blocking the input tables to get a candidate set</b>

<b>3. Matching the tuple pairs in the candidate set</b>

# Read input tables

We begin by loading the input tables. We need to provide the path where we have stored the table.

In [88]:
#Get the paths
path_google = "/home/gaurav/DS/stage3/csv_data/" + "playstore_formatted.csv"
path_apple = "/home/gaurav/DS/stage3/csv_data/" + "appstore_formatted.csv"

print path_google
print path_apple

/home/gaurav/DS/stage3/csv_data/playstore_formatted.csv
/home/gaurav/DS/stage3/csv_data/appstore_formatted.csv


In [89]:
# Load csv files as dataframes and set the key attribute in the dataframe
google = em.read_csv_metadata(path_google, key='ID')
apple = em.read_csv_metadata(path_apple, key='ID')

print('Number of tuples in table "google:" ' + str(len(google)))
print('Number of tuples in table "apple:" ' + str(len(apple)))
print('Number of tuples in "google" X "apple" (i.e the cartesian product): ' + str(len(google)*len(apple)))

Number of tuples in table "google:" 3789
Number of tuples in table "apple:" 3697
Number of tuples in "google" X "apple" (i.e the cartesian product): 14007933


In [90]:
#Printing few rows
google.head(2)

,ID,name,category,developer,rating
0,a1,Solitaire - Klondike,Card,Kiwi Solitaire,4.3
1,a2,slither.io,Action,Lowtech Studios,4.3


In [91]:
apple.head(2)

,ID,name,category,developer,rating
0,b1,Candy Crush Saga,Entertainment,King,4.70807
1,b2,Super Mario Run,Action,"Nintendo Co., Ltd.",2.97468


In [92]:
# Display the keys of the input tables
em.get_key(google), em.get_key(apple)

('ID', 'ID')

# Block Tables To Get Candidate Set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching. py_entitymatching provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that games with no overlap between name will not match. Also, there should be overlap between developer name as well. So we decide to apply blocking over game name and developer name:


In [93]:
# Blocking plan

# google, apple -- Overlap blocker [name] --------------------|---> candidate set 1
# candidate set 1 -- Overlap blocker [developer] -------------|---> candidate set 2

In [94]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
cand_1 = ob.block_tables(google, apple, 'name', 'name', 
                    l_output_attrs=['name', 'category', 'developer', 'rating'], 
                    r_output_attrs=['name', 'category', 'developer', 'rating'],
                    overlap_size=2, show_progress=False
                    )

# Again applying blocking using 'developer' attribute
cand_2 = ob.block_candset(cand_1, 'developer', 'developer', word_level=True, overlap_size=1, show_progress=False)

len(cand_2)

0%                          100%
[##############################] | ETA: 00:00:00 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:01


4307

# Match tuple pairs in candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes. This typically involves the following four steps:

    1. Sampling and labeling the candidate set
    2. Splitting the labeled data into development and evaluation set
    3. Selecting the best learning based matcher using the development set
    4. Evaluating the selected matcher using the evaluation set

## Labeling the candidate set

Our blocking step efficiently eliminated most of the obvious non matching tuple pairs. Hence we don't need to further sample the data. Complete candidate set obtained after blocking is used for labeling.

For labeling, we are converting candidate set back to .csv file and labeling the tuples manually. We are adding a column named 'label' and enter 0 for mismatch and 1 for match. 

In [95]:
#Converting to .csv file
cand_2.to_csv('cand_2.csv')

In [101]:
#Reading back the file after adding the labels
read_back = em.read_csv_metadata('cand_2.csv', 
                         key='_id',
                         ltable=google, rtable=apple, 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID')

len(read_back)

4307

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.


In [102]:
# Split the data in read_back into development set (I) and evaluation set (J)
IJ = em.split_train_test(read_back, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher

Selecting the best learning-based matcher typically involves the following steps:

    1. Creating a set of learning-based matchers
    2. Creating features
    3. Converting the development set into feature vectors
    4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers


In [103]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features
Next, we need to create a set of features for the development set. py_entitymatching provides a way to automatically generate features based on the attributes in the input tables.

In [104]:
# Generate features
feature_table = em.get_features_for_matching(google, apple)

### Converting the development set to feature vectors

In [105]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='label',
                            show_progress=False)

In [106]:
# Display first few rows
H.head(3)

,_id,ltable_ID,rtable_ID,ID_ID_lev_dist,ID_ID_lev_sim,ID_ID_jar,ID_ID_jwn,ID_ID_exm,ID_ID_jac_qgm_3_qgm_3,name_name_jac_qgm_3_qgm_3,...,category_category_sw,developer_developer_jac_qgm_3_qgm_3,developer_developer_cos_dlm_dc0_dlm_dc0,developer_developer_jac_dlm_dc0_dlm_dc0,developer_developer_mel,developer_developer_lev_dist,developer_developer_lev_sim,developer_developer_nmw,developer_developer_sw,label
3018,30692,a2306,b2285,4,0.2,0.466667,0.466667,0,0.0,0.243590,...,2.0,0.063492,0.192450,0.090909,0.504762,44.0,0.120000,-30.0,6.0,0
1364,11078,a1501,b1178,4,0.2,0.466667,0.466667,0,0.0,0.100000,...,1.0,0.137931,0.353553,0.200000,0.473430,19.0,0.173913,-13.0,4.0,0
3817,37880,a734,b3118,5,0.0,0.483333,0.483333,0,0.0,0.025641,...,1.0,0.160000,0.353553,0.200000,0.312865,15.0,0.210526,-9.0,4.0,0


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. We use five fold cross validation and use 'precision', 'recall' and 'f1' metric to select the best matcher.

In [107]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='precision', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0xa881b8cc>,5,0.979592,0.972603,0.967213,0.945205,1.000000,0.972923
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0xa83c8a8c>,5,1.000000,1.000000,1.000000,0.985294,1.000000,0.997059
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0xa83c8acc>,5,0.895833,1.000000,0.962264,0.981132,0.918367,0.951519
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0xa83c8aac>,5,0.903846,0.985294,1.000000,0.985075,0.932203,0.961284
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0xa83c8a6c>,5,0.886792,0.957746,0.904762,0.984848,0.948276,0.936485
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0xa83c8aec>,5,0.405172,0.511278,0.468750,0.485294,0.528846,0.479868


In [108]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='recall', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0xa881b8cc>,5,1.000000,0.986111,0.936508,0.985714,0.983607,0.978388
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0xa83c8a8c>,5,1.000000,0.972222,0.920635,0.957143,0.967213,0.963443
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0xa83c8acc>,5,0.895833,0.861111,0.809524,0.742857,0.737705,0.809406
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0xa83c8aac>,5,0.979167,0.930556,0.920635,0.942857,0.901639,0.934971
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0xa83c8a6c>,5,0.979167,0.944444,0.904762,0.928571,0.901639,0.931717
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0xa83c8aec>,5,0.979167,0.944444,0.952381,0.942857,0.901639,0.944098


In [109]:
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
        k=5,
        target_attr='label', metric='f1', random_state=0)
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<py_entitymatching.matcher.dtmatcher.DTMatcher object at 0xa881b8cc>,5,0.989691,0.979310,0.951613,0.965035,0.991736,0.975477
1,RF,<py_entitymatching.matcher.rfmatcher.RFMatcher object at 0xa83c8a8c>,5,1.000000,0.985915,0.958678,0.971014,0.983333,0.979788
2,SVM,<py_entitymatching.matcher.svmmatcher.SVMMatcher object at 0xa83c8acc>,5,0.895833,0.925373,0.879310,0.845528,0.818182,0.872845
3,LinReg,<py_entitymatching.matcher.linregmatcher.LinRegMatcher object at 0xa83c8aac>,5,0.940000,0.957143,0.958678,0.963504,0.916667,0.947198
4,LogReg,<py_entitymatching.matcher.logregmatcher.LogRegMatcher object at 0xa83c8a6c>,5,0.930693,0.951049,0.904762,0.955882,0.924370,0.933351
5,NaiveBayes,<py_entitymatching.matcher.nbmatcher.NBMatcher object at 0xa83c8aec>,5,0.573171,0.663415,0.628272,0.640777,0.666667,0.634460


We observe that the best matcher (RF) is getting us the best precision and f1. Recall is also high and is only slightly less than that for DecisionTree. So, we select <b>RF matcher</b> and now we can proceed on to evaluating the best matcher on the unseen data (the evaluation set).

## Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:

    1. Converting the evaluation set to feature vectors
    2. Training matcher using the feature vectors extracted from the development set
    3. Predicting the evaluation set using the trained matcher
    4. Evaluating the predicted matches
    

### Converting the evaluation set to feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set).


In [110]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='label', show_progress=False)

### Training matcher, predicting the evaluation set and evaluating predicted matches for each matcher

We train the matchers using all of the feature vectors from the development set.

We predict the matches for the evaluation set (using the feature vectors extracted from it).

Finally, we evaluate the accuracy of predicted outputs.

#### 1. Decision Tree

In [111]:
# Train using feature vectors from I 
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = dt.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 98.87% (175/177)
Recall : 98.31% (175/178)
F1 : 98.59%
False positives : 2 (out of 177 positive predictions)
False negatives : 3 (out of 1116 negative predictions)


#### 2. RF

In [112]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 99.43% (175/176)
Recall : 98.31% (175/178)
F1 : 98.87%
False positives : 1 (out of 176 positive predictions)
False negatives : 3 (out of 1117 negative predictions)


#### 3. SVM

In [113]:
# Train using feature vectors from I 
svm.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = svm.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 96.77% (150/155)
Recall : 84.27% (150/178)
F1 : 90.09%
False positives : 5 (out of 155 positive predictions)
False negatives : 28 (out of 1138 negative predictions)


#### 4. LinReg

In [114]:
# Train using feature vectors from I 
ln.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = ln.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.08% (166/171)
Recall : 93.26% (166/178)
F1 : 95.13%
False positives : 5 (out of 171 positive predictions)
False negatives : 12 (out of 1122 negative predictions)


#### 5. LogReg

In [115]:
# Train using feature vectors from I 
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = lg.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 91.76% (167/182)
Recall : 93.82% (167/178)
F1 : 92.78%
False positives : 15 (out of 182 positive predictions)
False negatives : 11 (out of 1111 negative predictions)


#### 6. NaiveBayes

In [116]:
# Train using feature vectors from I 
nb.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
       target_attr='label')

# Predict on L 
predictions = nb.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
              append=True, target_attr='predicted', inplace=False)

# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 58.68% (169/288)
Recall : 94.94% (169/178)
F1 : 72.53%
False positives : 119 (out of 288 positive predictions)
False negatives : 9 (out of 1005 negative predictions)


### Best matcher

RF matcher performed best in the evaluation set as well. Here are the statistics:

Precision : 99.43% (175/176)

Recall : 98.31% (175/178)

F1 : 98.87%

False positives : 1 (out of 176 positive predictions)

False negatives : 3 (out of 1117 negative predictions)